# Machine Learning for Demand Forecasting

### Use case - predicting the demand for items at a libary

In [158]:
# sfOptions = {
#   "sfURL" : "datalytyx.east-us-2.azure.snowflakecomputing.com",
#   "sfAccount" : "datalytyx",
#   "sfUser" : "WILLHOLTAM",
#   "sfPassword" : "04MucSfLV",
#   "sfRole": "DATABRICKS",
#   "sfDatabase" : "DATABRICKS_DEMO",
#   "sfSchema" : "SEATTLE_LIBRARY",
#   "sfWarehouse" : "DATASCIENCE_WH"
# }
# SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

In [159]:
# #spark.conf.set("spark.executor.cores",2)

# df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
#   .options(**sfOptions) \
#   .option("query", """select * from library_collection_inventory where reportdate in ('2017-09-01T00:00:00','2017-10-01T00:00:00', '2017-11-01T00:00:00', '2017-12-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2018-02-01T00:00:00', '2018-02-01T00:00:00', '2018-03-01T00:00:00', '2018-04-01T00:00:00', '2018-05-01T00:00:00', '2018-06-01T00:00:00', '2018-07-01T00:00:00') """) \
#   .load().limit(1000)

In [160]:
# # Create a view or table

# temp_table_name = "library_collection_inventory"

# df.createOrReplaceTempView(temp_table_name)

In [194]:
# Import Libraries
import numpy as np
import pandas as pd
import nltk  # Has to be added through Workspaces/ attach library to cluster
import more_itertools
import re
import os
import codecs
import mpld3
from snowflake.sqlalchemy import URL
from nltk.stem.snowball import SnowballStemmer
from sqlalchemy import create_engine
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import feature_extraction
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import Pipeline, FeatureUnion

In [162]:
# df_pandas = df.toPandas()  # Create pandas dataframe to work within python when in Databricks

In [163]:
sf_account = "datalytyx"
sf_user = "WILLHOLTAM"
sf_pwd = "04MucSfLV"
# sf_user = "CHRISSCHON"
# sf_pwd = "UpsetSheep7"
sf_role = "DATABRICKS"
sf_db = "DATABRICKS_DEMO"
sf_schema = "SEATTLE_LIBRARY"
sf_wh = "DATASCIENCE_WH"
sf_region = "east-us-2.azure"

In [164]:
engine = create_engine(URL(
    user = sf_user,
    password = sf_pwd,
    account = sf_account,
    region = sf_region,
    database = sf_db,
    schema = sf_schema,
    warehouse = sf_wh,
    role = sf_role,
))

# engine = create_engine(connection_string)

In [165]:
features = pd.read_sql_query("select * from library_collection_inventory where reportdate in ('2017-09-01T00:00:00','2017-10-01T00:00:00', '2017-11-01T00:00:00', '2017-12-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2018-02-01T00:00:00', '2018-02-01T00:00:00', '2018-03-01T00:00:00', '2018-04-01T00:00:00', '2018-05-01T00:00:00', '2018-06-01T00:00:00', '2018-07-01T00:00:00') limit 1000", engine)

In [166]:
nltk.download('stopwords')  # Common words to ignore
nltk.download('punkt')  # Punkt Sentence Tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WilliamHoltam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WilliamHoltam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [167]:
# df_pandas.dropna(axis=1, how='all')  # Drop the columns where all of the elements are missing values
# df_pandas.dropna(axis=0, how='any')  # Drop the rows where any of the elements are missing values

In [168]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [169]:
# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")

In [170]:
class NoneReplacer(TransformerMixin, BaseEstimator):
    
    """
    Transformer changes Nonetype values into numpy NaN values.
    """
    
    def __init__(self):
        pass

    def fit(self, X, y = None):
#        X_fitted = X.where(X == None)
        return self
        
    def transform(self, X):
        #["" if item is None else str(item) for item in X.select_dtypes(include='object')]
        assert isinstance(X, pd.DataFrame)
        
        X.fillna(value = pd.np.nan, inplace=True)
        
        return X

In [171]:
initiated_class = NoneReplacer()

initiated_class.fit(features)

df_pandas_fitter = initiated_class.transform(features)
# df_pandas_fitter

In [172]:
class EmptyColumnRemover(TransformerMixin, BaseEstimator):
    
    """
    Transformer drops empty columns
    """
    
    def __init__(self):
        pass
    
    def fit(self, X, y = None):  # has to take an optional y for pipelines
        
        """
        Calculates the number of missing values that corresponds to the threshold.
        Detects and labels columns with equal to or greater than numbers of missing values than the threshold.
        """
        
        self.drop_columns = features.isna().sum()[features.isna().sum() >= X.shape[0]].index  # Calculates pd.series with column lables as indecies
        
        return self
        
    def transform(self, X):
        
        """
        Drops columns containing empty values.
        """
        
        assert isinstance(X, pd.DataFrame)
        
        return X.drop(columns = self.drop_columns)

In [173]:
initiated_class = EmptyColumnRemover()

initiated_class.fit(X=df_pandas_fitter)

df_pandas_fitter1 = initiated_class.transform(df_pandas_fitter)
# df_pandas_fitter1

In [174]:
class AnyNaNRowRemover(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        pass
        
    def fit(self, X, y = None):  # has to take an optional y for pipelines
        
        """
        Calculates the number of missing values the corresponds to the threshold.
        Detects and labels columns with more missing values that the threshold. 
        """
        return self
    
    def transform(self, X):
        
        assert isinstance(X, pd.DataFrame)
        
        return X.dropna(axis=0, how='any')

In [175]:
initiated_class = AnyNaNRowRemover()

initiated_class.fit(df_pandas_fitter1)

df_pandas_fitter2 = initiated_class.transform(df_pandas_fitter1)

In [186]:
class TokenizeAndStemer(TransformerMixin, BaseEstimator):
    
    def __init__(self, default_column = 'subjects'):
        self.default_column = default_column
        pass
    
    def tokenize_and_stem(self, X):
        
        tokens = [word for sent in nltk.sent_tokenize(X) for word in nltk.word_tokenize(sent)]
        
        filtered_tokens = []
        [filtered_tokens.append(token) if re.search('[a-zA-Z]', token) else token for token in tokens]
        
        stems = [stemmer.stem(t) for t in filtered_tokens]
        
        return stems
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):

        totalvocab_stemmed = []
        
        allwords_stemmed = [self.tokenize_and_stem(str(i)) for i in X[self.default_column].tolist()] #for each item in 'synopses', tokenize/stem
        totalvocab_stemmed.extend(allwords_stemmed)
        totalvocab_stemmed = list(more_itertools.collapse(totalvocab_stemmed))
        
        return totalvocab_stemmed

# How to implement in class above???? OR do I have to implement it in a separate class...  
# allwords_stemmed = [tokenize_and_stem(str(i)) for i in subjects] #for each item in 'synopses', tokenize/stem
# totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
# totalvocab_stemmed = list(more_itertools.collapse(totalvocab_stemmed)) 

In [187]:
initiated_class = TokenizeAndStemer()

initiated_class.fit(df_pandas_fitter2)

df_pandas_fitter3 = initiated_class.transform(df_pandas_fitter2)

In [184]:
class TokenizeOnly(TransformerMixin, BaseEstimator):
    
    def __init__(self, default_column = 'subjects'):
        self.default_column = default_column
        pass

    def tokenize_only(self, X):
        
        # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
        tokens = [word.lower() for sent in nltk.sent_tokenize(X) for word in nltk.word_tokenize(sent)]
        filtered_tokens = []
        
        # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
        [filtered_tokens.append(token) if re.search('[a-zA-Z]', token) else token for token in tokens]
                
        return filtered_tokens
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):

        totalvocab_tokenized = []
    
        allwords_tokenized = [self.tokenize_only(str(i)) for i in X[self.default_column].tolist()] #for each item in 'synopses', tokenize/stem
        totalvocab_tokenized.extend(allwords_tokenized)
        totalvocab_tokenized = list(more_itertools.collapse(totalvocab_tokenized))
        
        return totalvocab_tokenized

In [188]:
initiated_class = TokenizeOnly()

initiated_class.fit(df_pandas_fitter2)

df_pandas_fitter3 = initiated_class.transform(df_pandas_fitter2)

In [203]:
union = FeatureUnion([("tas", TokenizeAndStemer()),
                      ("to", TokenizeOnly())])

test = union.fit_transform(df_pandas_fitter2)
print(test.shape)

(19114,)


In [ ]:
# class TokenizeOnly(TransformerMixin, BaseEstimator):

#     def __init__(self, default_column = 'subjects'):
#         self.default_column = default_column
#         pass
    
#      def fit(self, X, y = None):
#         return self
    
#     def transform(self, X):
#         vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
# print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

In [189]:
pre_processing_pipeline = Pipeline([
    ('nr', NoneReplacer()),
    ('ecr', EmptyColumnRemover()),
    ('anrr', AnyNaNRowRemover())
])

In [190]:
df_cleaned = pre_processing_pipeline.fit_transform(X = features)

In [191]:
tokenize_and_stem_pipeline = Pipeline([
    ('tas', TokenizeAndStemer())
])

list_tokenized_and_stemmed = tokenize_and_stem_pipeline.fit_transform(X = df_cleaned)

In [192]:
tokenize_only_pipeline = Pipeline([   
    ('to', TokenizeOnly())
])

list_tokenized = tokenize_only_pipeline.fit_transform(X = df_cleaned)

In [122]:
# def tokenize_and_stem(X, default_column = 'subjects'):
#     tokens = [word for sent in nltk.sent_tokenize(X[default_column]) for word in nltk.word_tokenize(sent)]
        
#     filtered_tokens = []
#     [filtered_tokens.append(token) if re.search('[a-zA-Z]', token) else token for token in tokens]
    
#     stems = [stemmer.stem(t) for t in filtered_tokens]
        
#     return stems

# tokenize_and_stem(df_pandas_fitter2)

In [121]:
# def tokenize_and_stem(text):
#     # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
#     tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
#     filtered_tokens = []
#     # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
#     for token in tokens:
#         if re.search('[a-zA-Z]', token):
#             filtered_tokens.append(token)
#     stems = [stemmer.stem(t) for t in filtered_tokens]
#     return stems

# tokenize_and_stem(df_pandas_fitter2['subjects'])

In [54]:
# def tokenize_and_stem(text):
#     # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
#     tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
#     filtered_tokens = []
#     # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
#     for token in tokens:
#         if re.search('[a-zA-Z]', token):
#             filtered_tokens.append(token)
#     stems = [stemmer.stem(t) for t in filtered_tokens]
#     return stems

# allwords_stemmed = [tokenize_and_stem(str(i)) for i in subjects] #for each item in 'synopses', tokenize/stem
# totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
# totalvocab_stemmed = list(more_itertools.collapse(totalvocab_stemmed)) 

In [826]:
# subjects = df_pandas['subjects']

# Define a tokenizer and stemmer which returns the set of stems in the text that it is passed

# #class TextTransform()
# def tokenize_and_stem(text):
#     # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
#     tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
#     filtered_tokens = []
#     # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
#     for token in tokens:
#         if re.search('[a-zA-Z]', token):
#             filtered_tokens.append(token)
#     stems = [stemmer.stem(t) for t in filtered_tokens]
#     return stems


# def tokenize_only(text):
#     # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
#     tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
#     filtered_tokens = []
#     # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
#     for token in tokens:
#         if re.search('[a-zA-Z]', token):
#             filtered_tokens.append(token)
#     return filtered_tokens

# totalvocab_stemmed = []
# totalvocab_tokenized = []

# allwords_stemmed = [tokenize_and_stem(str(i)) for i in subjects] #for each item in 'synopses', tokenize/stem
# totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
# totalvocab_stemmed = list(more_itertools.collapse(totalvocab_stemmed)) 

# allwords_tokenized = [tokenize_only(str(i)) for i in subjects]
# totalvocab_tokenized.extend(allwords_tokenized)
# totalvocab_tokenized = list(more_itertools.collapse(totalvocab_tokenized)) 

In [827]:
# totalvocab_stemmed = []
# totalvocab_tokenized = []
# for i in subjects_c:
#     allwords_stemmed = tokenize_and_stem(str(i)) #for each item in 'synopses', tokenize/stem
#     totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
#     allwords_tokenized = tokenize_only(str(i))
#     totalvocab_tokenized.extend(allwords_tokenized)
# totalvocab_tokenized

In [828]:
# class ModelTransformer(TransformerMixin):
#     def __init__(self, model):
#         self.model = model

#     def fit(self, *args, **kwargs):
#         self.model.fit(*args, **kwargs)
#         return self

#     def transform(self, X, **transform_params):
#         return pd.DataFrame(self.model.predict(X))

# pipeline = Pipeline([
#     ('cluster', ModelTransformer(KMeans_foo(3))),
#     ('binarize', LabelBinarizer())
# ])

In [829]:
# df_pandas.loc[:,'PUBLICATIONYEAR'] = df_pandas.loc[:,'PUBLICATIONYEAR'].str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)

# _______________________________________________________________________________
# Functions for transforming the data
# def processing(df):
#     df['PUBLICATIONYEAR'] = df['PUBLICATIONYEAR'].str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)
#     [str(item) for item in df['SUBJECTS'] if item is None]

# processing(df_pandas)
    
    # def ext_date_fun(input, output):
#   output = input.str.extractstr.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)
  
# def fix_none_fun(sub):
#   [str(item) for item in sub if item is None]
# _______________________________________________________________________________

# # Attempt at putting the functions into classes
# class PublicationYearCleaner(object):
#   """Preprocessing: This class cleans the Publication Year Column"""
#   def __init__(self, data):
#     self.raw = data
  
#   def ext_date(self, pub_year):
#      pub_year = pub_year.str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)

# class SubjectsCleaner(objct)
#   def fix_none(self, string):
#     if string is None:
#       return ''
#     return str(string)
# # ________________________________________________________________________________


In [830]:
# df_pandas.loc[:,'PUBLICATIONYEAR'] = df_pandas.loc[:,'PUBLICATIONYEAR'].str.extract(r'(^|)*([0-9]{4})\s*(|$)', expand=True)



In [831]:
# n = 1000  # Number of rows to analyse

# titles = df_pandas.loc[0:n,'TITLE'].values.tolist()
# subjects = df_pandas.loc[0:n,'SUBJECTS'].values.tolist()
# author = df_pandas.loc[0:n, 'AUTHOR'].values.tolist()

In [832]:
# class ModelTransformer(TransformerMixin):
#     def __init__(self, model):
#         self.model = model

#     def fit(self, *args, **kwargs):
#         self.model.fit(*args, **kwargs)
#         return self

#     def transform(self, X, **transform_params):
#         return pd.DataFrame(self.model.predict(X))

In [833]:
# df_pandas = df_pandas.join(pd.get_dummies(df_pandas.loc[:,'ITEMTYPE']), how='inner')
# df_pandas = df_pandas.join(pd.get_dummies(df_pandas.loc[:,'ITEMCOLLECTION']), how='inner')
# df_pandas = df_pandas.join(pd.get_dummies(df_pandas.loc[:,'ITEMLOCATION']), how='inner')
# list(df_pandas.columns.values)


In [834]:
# #use extend so it's a big flat list of vocab
# totalvocab_stemmed = []
# totalvocab_tokenized = []
# for i in df_pandas['subjects_c']:
#     allwords_stemmed = tokenize_and_stem(str(i)) #for each item in 'synopses', tokenize/stem
#     totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
#     allwords_tokenized = tokenize_only(str(i))
#     totalvocab_tokenized.extend(allwords_tokenized)

In [835]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 13598 items in vocab_frame


In [836]:
print(vocab_frame.head())

              words
allon         allon
gabriel     gabriel
fictiti  fictitious
charact   character
fiction     fiction


In [837]:
#define vectorizer parameters
vectorizer_pipe = Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=5, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3)))])

%time tfidf_matrix = vectorizer_pipe.fit_transform(subjects)  # fit the vectorizer to synopses

print(tfidf_matrix.shape)

Wall time: 450 ms
(1000, 737)


In [838]:
terms = tfidf_vectorizer.get_feature_names()
dist = 1 - euclidean_distances(tfidf_matrix)

cluster_pipe = Pipeline(steps=[('cluster', KMeans(n_clusters = 5, random_state=3425))])
%time cluster_pipe.fit(tfidf_matrix)
clusters = cluster_pipe.named_steps['cluster'].labels_.tolist()

NameError: name 'tfidf_vectorizer' is not defined

In [ ]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(cluster_pipe,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = cluster_pipe.named_steps['cluster'].labels_.tolist()

In [ ]:
books = { 'title': titles, 'author': author, 'subjects': subjects_c, 'cluster': clusters }

frame = pd.DataFrame(books, index = [clusters] , columns = ['title', 'author', 'cluster'])

In [ ]:
# frame.columns
frame.cluster.value_counts() #number of books per cluster (clusters from 0 to 4)

In [ ]:
from __future__ import print_function

print(ind)
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = cluster_pipe.named_steps['cluster'].cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
        print() #add whitespace
        print() #add whitespace
    
    for ind in order_centroids[i, :3]: #replace 6 with n words per cluster
        x += ' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=','
    
#    print("Cluster %d titles:" % i, end='')
#    for title in frame.loc[i,'title'].values.tolist():
#        print(' %s,' % title, end='')

In [ ]:
# import os  # for os.path.basename

# import matplotlib.pyplot as plt
# import matplotlib as mpl

# from sklearn.manifold import MDS

# MDS()

# # convert two components as we're plotting points in a two-dimensional plane
# # "precomputed" because we provide a distance matrix
# # we will also specify `random_state` so the plot is reproducible.
# mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

# pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

# xs, ys = pos[:, 0], pos[:, 1]

In [ ]:
dummies = pd.get_dummies(frame.cluster)
dummies.columns = dummies.columns.astype(str)
list(dummies.columns.values)


In [ ]:
# dummies.rename(index=str, columns={"0": "Juvenile Literature", "1": "Music (Country)", "2": "Mystery", "3": "Music (Rock)", "4": "Comic Books"})
dummies.columns = ["Drama / Film / Rock", "Juvinile Mystery", "United States / Biography", "Juvinile Literature / Biography", "Comic Books"]
list(dummies.columns.values)

In [ ]:
#frame
df_pandas_dummies = df_pandas.join(dummies, how='left')
df_pandas_dummies
list(df_pandas_dummies.columns.values)

In [ ]:
prediction = df_pandas.loc[(n+1):,:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_pandas_dummies, dummies, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)